In [1]:
import numpy as np
import torch

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)

In [6]:
def model(x):
    return x @ w.t() + b

In [7]:
pred = model(inputs)
def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.nume1()
loss = mse(pred,targets)
print(loss)

AttributeError: 'Tensor' object has no attribute 'nume1'